Dependencias

In [2]:
!pip install keras-tuner


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
from tensorflow.keras.layers import Dense, Dropout, Input
import tensorflow as tf
from tensorflow.keras.models import Sequential
from kerastuner.tuners import RandomSearch
import numpy as np
import os
from tensorflow.keras.optimizers import Adam
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import joblib

In [ ]:
possible_paths = [
    '../analisis_exploratorios_tp3/',  
    '../../analisis_exploratorios_tp3/'  
]

data_path = None
for path in possible_paths:
    if os.path.exists(f"{path}train.csv"):
        data_path = path
        break

if data_path:
    print(f" Archivos CSV encontrados en: {data_path}")
    train = pd.read_csv(f'{data_path}train.csv', index_col=0, parse_dates=True)
    validation = pd.read_csv(f'{data_path}validation.csv', index_col=0, parse_dates=True)
    test = pd.read_csv(f'{data_path}test.csv', index_col=0, parse_dates=True)
else:
    print(" No se encontraron los archivos CSV en ninguna ruta")

print(f" Datos cargados exitosamente:")
print(f"   Train: {train.shape}")
print(f"   Validation: {validation.shape}")
print(f"   Test: {test.shape}")
print(f"   Columnas disponibles: {len(train.columns)} columnas")

 Archivos CSV encontrados en: ../analisis_exploratorios_tp3/
 Datos cargados exitosamente:
   Train: (1110, 48)
   Validation: (370, 48)
   Test: (370, 48)
   Columnas disponibles: 48 columnas


In [4]:
mapper_rn = ColumnTransformer([
    ('scale_btc_h', StandardScaler(), ['btc_h']),
    ('scale_btc_l', StandardScaler(), ['btc_l']),
    ('scale_btc_o', StandardScaler(), ['btc_o']),
    ('scale_btc_v', StandardScaler(), ['btc_v']),
    
    ('scale_btc_prev_c1', StandardScaler(), ['btc_prev_c1']),
    ('scale_btc_prev_c2', StandardScaler(), ['btc_prev_c2']),
    ('scale_btc_prev_c3', StandardScaler(), ['btc_prev_c3']),
    ('scale_btc_prev_c4', StandardScaler(), ['btc_prev_c4']),
    ('scale_btc_prev_c5', StandardScaler(), ['btc_prev_c5']),
    ('scale_btc_prev_c6', StandardScaler(), ['btc_prev_c6']),
    ('scale_btc_prev_c7', StandardScaler(), ['btc_prev_c7']),
   
    ('scale_fng_value', StandardScaler(), ['fng_value']),
    ('scale_bitcoin', StandardScaler(), ['bitcoin']),
    ('scale_is_month_start', StandardScaler(), ['is_month_start']),
    ('scale_is_month_end', StandardScaler(), ['is_month_end']),
    
    ('scale_btc_pct_prev1', StandardScaler(), ['btc_pct_prev1']),
    ('scale_btc_pct_prev2', StandardScaler(), ['btc_pct_prev2']),
    ('scale_btc_pct_prev3', StandardScaler(), ['btc_pct_prev3']),
    ('scale_btc_pct_prev4', StandardScaler(), ['btc_pct_prev4']),
    ('scale_btc_pct_prev5', StandardScaler(), ['btc_pct_prev5']),
    ('scale_btc_pct_prev6', StandardScaler(), ['btc_pct_prev6']),
    ('scale_btc_pct_prev7', StandardScaler(), ['btc_pct_prev7']),
    
    ('scale_RSI_14', StandardScaler(), ['RSI_14']),
    ('scale_BBL_20_2.0_2.0', StandardScaler(), ['BBL_20_2.0_2.0']),
    ('scale_BBM_20_2.0_2.0', StandardScaler(), ['BBM_20_2.0_2.0']),
    ('scale_BBU_20_2.0_2.0', StandardScaler(), ['BBU_20_2.0_2.0']),
    ('scale_BBB_20_2.0_2.0', StandardScaler(), ['BBB_20_2.0_2.0']),
    ('scale_BBP_20_2.0_2.0', StandardScaler(), ['BBP_20_2.0_2.0']),
    ('scale_MACD_12_26_9', StandardScaler(), ['MACD_12_26_9']),
    ('scale_MACDh_12_26_9', StandardScaler(), ['MACDh_12_26_9']),
    ('scale_MACDs_12_26_9', StandardScaler(), ['MACDs_12_26_9']),
    ('scale_ATRr_14', StandardScaler(), ['ATRr_14']),

    ('scale_sp500_c', StandardScaler(), ['sp500_c']),
    ('scale_dowj_c', StandardScaler(), ['dowj_c']),
    ('scale_nasdaq_c', StandardScaler(), ['nasdaq_c']),
    ('scale_eurostoxx_c', StandardScaler(), ['eurostoxx_c']),
    ('scale_nikkei_c', StandardScaler(), ['nikkei_c']),
 
    ('ohe_day', OneHotEncoder(handle_unknown='ignore'), ['day']),
    ('ohe_month', OneHotEncoder(handle_unknown='ignore'), ['month']),
    ('ohe_year', OneHotEncoder(handle_unknown='ignore'), ['Year'])
])

mapper_rn.fit(train)
train_transformed = mapper_rn.transform(train)
validation_transformed = mapper_rn.transform(validation)
test_transformed = mapper_rn.transform(test)
train_df = pd.DataFrame(train_transformed, columns=mapper_rn.get_feature_names_out(), index=train.index)

features = [
    'btc_c', 'btc_h', 'btc_l', 'btc_o', 'btc_v',
    'sp500_c', 'dowj_c', 'nasdaq_c', 'eurostoxx_c', 'nikkei_c',
    'day', 'month', 'Year',
    'btc_prev_c1', 'btc_prev_c2', 'btc_prev_c3', 'btc_prev_c4',
    'btc_prev_c5', 'btc_prev_c6', 'btc_prev_c7',
    'btc_pct_prev1', 'bitcoin',
    'is_month_start', 'is_month_end', 'fng_value',  
    'btc_pct_prev2', 'btc_pct_prev3', 'btc_pct_prev4', 'btc_pct_prev5', 
    'btc_pct_prev6', 'btc_pct_prev7', 'RSI_14', 'BBL_20_2.0_2.0', 
    'BBM_20_2.0_2.0', 'BBU_20_2.0_2.0', 'BBB_20_2.0_2.0', 'BBP_20_2.0_2.0', 
    'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'ATRr_14'
]
features = [f for f in features if f in train.columns]


In [ ]:
def build_model_simplified(hp):
    model = Sequential()
    model.add(Input(shape=(num_features,)))
    
    hp_units = hp.Choice('units_1', values=[60, 80, 96, 128])
    model.add(Dense(units=hp_units, activation='relu'))

    hp_dropout = hp.Choice('dropout_1', values=[0.2, 0.3, 0.4])
    model.add(Dropout(hp_dropout))
    
    model.add(Dense(1))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 5e-4, 1e-4])
    
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                  loss='mse',
                  metrics=['mae'])
    return model

#Configurar el Buscador (RandomSearch)
tuner = RandomSearch(
    build_model_simplified,
    objective='val_loss',      
    max_trials=30,            
    executions_per_trial=1,    
    directory='hp_tuning_dir',
    project_name='btc_tuning',
    overwrite=True  
)

# Ejecutar la Búsqueda
print("\n--- Iniciando Búsqueda de Hiperparámetros ---")
stop_early = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

tuner.search(
    X_train_scaled, y_train_scaled,
    epochs=50,                
    validation_data=(X_val_scaled, y_val_scaled),
    callbacks=[stop_early],
    verbose=1               
)

# Obtener y Mostrar los Mejores Resultados
print(" MEJORES HIPERPARÁMETROS ENCONTRADOS ")

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"• Neuronas en capa oculta: {best_hps.get('units_1')}")
print(f"• Porcentaje de Dropout:   {best_hps.get('dropout_1')}")
print(f"• Tasa de Aprendizaje:     {best_hps.get('learning_rate')}")

Trial 30 Complete [00h 00m 03s]
val_loss: 0.030005892738699913

Best val_loss So Far: 0.013575674034655094
Total elapsed time: 00h 01m 27s

 MEJORES HIPERPARÁMETROS ENCONTRADOS 
• Neuronas en capa oculta: 128
• Porcentaje de Dropout:   0.4
• Tasa de Aprendizaje:     0.001


In [25]:
X_train_full = mapper_rn.transform(train[features])
X_val_full   = mapper_rn.transform(validation[features])

#  Función para crear el modelo ( reinicia en cada loop)
def crear_modelo_base(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mape'])
    return model

In [26]:
targets = [f"NextClose_BTC{i}" for i in range(1, 8)] # Del 1 al 7
resultados = {}

for target in targets:
    print(f"\n======== Procesando {target} ========")

    # 1. Seleccionar y (Cambia en cada iteración)
    y_train_raw = train[target]
    y_val_raw   = validation[target]

    # 2. Máscara de limpieza (Cambia porque los NaN del target son distintos cada día)
    mask_train = (~np.isnan(X_train_full).any(axis=1)) & np.isfinite(y_train_raw)
    mask_val   = (~np.isnan(X_val_full).any(axis=1)) & np.isfinite(y_val_raw)

    # 3. Aplicar filtros
    X_train_curr = X_train_full[mask_train]
    y_train_curr = y_train_raw[mask_train]

    X_val_curr = X_val_full[mask_val]
    y_val_curr = y_val_raw[mask_val]

    # 4. Escalado (Obligatorio repetirlo porque los datos filtrados cambian)
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train_curr)
    X_val_scaled   = scaler_X.transform(X_val_curr)

    y_train_scaled = scaler_y.fit_transform(y_train_curr.values.reshape(-1, 1))
    y_val_scaled   = scaler_y.transform(y_val_curr.values.reshape(-1, 1))

    # 5. Crear modelo NUEVO (Importante: resetear pesos)
    model = crear_modelo_base(X_train_scaled.shape[1])

    # 6. Entrenar
    history = model.fit(
        X_train_scaled, y_train_scaled,
        validation_data=(X_val_scaled, y_val_scaled),
        epochs=200, 
        batch_size=15,
        verbose=0 
    )

    y_pred_scaled = model.predict(X_val_scaled, verbose=0).flatten()
    y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

    mape = np.mean(np.abs((y_val_curr - y_pred) / y_val_curr)) * 100
    print(f"Target: {target} | MAPE: {mape:.2f}%")
    
    resultados[target] = mape


======== Procesando NextClose_BTC1 ========
Target: NextClose_BTC1 | MAPE: 3.14%

======== Procesando NextClose_BTC2 ========
Target: NextClose_BTC2 | MAPE: 3.63%

======== Procesando NextClose_BTC3 ========
Target: NextClose_BTC3 | MAPE: 4.18%

======== Procesando NextClose_BTC4 ========
Target: NextClose_BTC4 | MAPE: 4.88%

======== Procesando NextClose_BTC5 ========
Target: NextClose_BTC5 | MAPE: 4.96%

======== Procesando NextClose_BTC6 ========
Target: NextClose_BTC6 | MAPE: 5.94%

======== Procesando NextClose_BTC7 ========
Target: NextClose_BTC7 | MAPE: 5.83%
